## 講習5 --- astropy.io.fitsの基本  


FITSデータの読み書きが可能なモジュールです。  元々、pyfitsと呼ばれていたものです。astropyに吸収されてそちらで統一的に管理されるようです。  
astropy.io.fitsの使い方は、 
http://docs.astropy.org/en/stable/io/fits/index.html
が非常に参考になります。  例が多く記載されているのでわかりやすいです。  

PyRAFでもFITSデータの読み書きはできるのですが、ndarrayとしてデータを読み出しておくとより自由な処理を行うことができます。例えば、IRAFには存在しない統計アルゴリズムを適用する、あるいは、matplotlibを用いた自由度の高い可視化などです。



In [1]:
from astropy.io import fits

まずastropy.ioからfitsをimportしておきます。

### 基本のgetdataとgetheader 
まずは簡単なデータ読み出し方法。** fits.getdata( )**と **fits.getheader( )**です。  主にインタラクティブなケースで使われます。  

まずはピクセル値をgetdataでndarrayとして読み出します。

In [2]:
data = fits.getdata('./data1/SUPA00317505.fits')  #  ドームフラットの生データ 

In [3]:
data

array([[15798, 15697, 14527, ...,  9985,  9981,  9976],
       [18091, 33233, 30777, ...,  9991,  9987,  9978],
       [15287, 31092, 25091, ...,  9986,  9981,  9979],
       ..., 
       [19053, 36375, 25385, ..., 10004, 10007,  9997],
       [16965, 34236, 24216, ...,  9998, 10001, 10000],
       [18178, 32768, 22645, ..., 10015, 10008, 10013]], dtype=uint16)

ndarrayとして読み出すので、numpyの関数が使えます。  

In [4]:
import numpy as np

In [5]:
np.median(data), np.std(data), np.min(data), np.max(data)

(18931.0, 1212.2635026788732, 9681, 38631)

オーバースキャンの部分も含めて統計をとっているので、標準偏差(np.std)が大きな値になっています。  
オーバースキャンの部分を除いて統計をとってみましょう。  

ndarrayなので、**[y, x]**の順に範囲を指定します。  
**[y_start:y_end, x_start:x_end]** のように範囲をしていします。値を省略すると、「最初から」 あるいは 「最後まで」となります。左右両方を省略すると「全部」です。    

In [6]:
subdata = data[:, :2045]
np.median(subdata), np.std(subdata), np.min(subdata), np.max(subdata)

(18935.0, 409.56087590112406, 9739, 38594)

上限値と下限値を適切に設定し、3-sigmaクリッップすると標準偏差がもう少し小さくなります。  

In [7]:
xx = np.where((subdata > 15000) & (subdata < 30000))
med = np.median(subdata[xx])
std = np.std(subdata[xx])

xx = np.where((subdata > med - 3 * std) & (subdata < med + 3 * std))
med = np.median(subdata[xx])
std = np.std(subdata[xx])
print ('{} {:.2f}'.format(med, std))   #   format()関数使ってみた

18934.0 213.65


次にgetheaderを使ってFITSヘッダを読み出します。  

In [8]:
header = fits.getheader('./data1/SUPA00317505.fits')

ここで、header あるいは print(header) をcellで実行すると、FITSヘッダの中身が表示されます。  

In [9]:
# header   # これをするとgithubでは表示が長くなるのでここではコメントアウト

最初の10項目だけを表示させます。

In [10]:
header[:10]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2080 / length of data axis 1                          
NAXIS2  =                 4100 / length of data axis 2                          
EXTEND  =                    F / FITS dataset may contain extensions            
BZERO   =              32768.0 / offset data range to that of unsigned short    
BSCALE  =                  1.0 / default scaling factor                         
BUNIT   = 'ADU     '           / Unit of original pixel value                   
BLANK   =               -32768 / Value used for NULL pixels                     

In [11]:
header['OBJECT']

'DOMEFLAT'

このようにヘッダのキーワードを指定して、その値を取り出すことができます。

** [小技] **  
実は、fits.getdata( )でヘッダも読み出すことができます。

In [12]:
tdata, thdr = fits.getdata('./data1/SUPA00317505.fits', header = True) 

In [13]:
thdr[:10]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2080 / length of data axis 1                          
NAXIS2  =                 4100 / length of data axis 2                          
EXTEND  =                    F / FITS dataset may contain extensions            
BZERO   =              32768.0 / offset data range to that of unsigned short    
BSCALE  =                  1.0 / default scaling factor                         
BUNIT   = 'ADU     '           / Unit of original pixel value                   
BLANK   =               -32768 / Value used for NULL pixels                     

### データの加工と書き出し   

このドームフラット1枚を使って、フラットを作成しましょう。 バイアス値がY方向に一様とする簡易版で行います。  
ここでは、IRAFを使わずnumpyを使って行います。

In [14]:
omed = np.median(data[:, 2049:])  #  オーバースキャン部分のメジアン
imed = np.median(data[:, :2048])  # 光が当たる部分のメジアン
data = ( data - omed ) / (imed - omed)  #  ゲタを引き、規格化

整数データを割り算した結果、浮動小数点数になります。  デフォルトでは64ビットになります。

In [15]:
data.dtype

dtype('float64')

無駄にファイルサイズが大きくなるので、32ビットに変更します。  

In [16]:
data = data.astype(np.float32) 

ヘッダのBITPIXは自動的に変更されます。

ヘッダのOBJECTの文字列を変更してみます。 

In [17]:
header['OBJECT'] = 'B_FLAT'

ヘッダキーワードのBLANKはデータが整数のときのみ有効です。  
それ以外のときにこの項目があると、アプリケーションによってはWarningがでます。  
ここでは削除しておきましょう。  

In [18]:
del header['BLANK']

このデータとヘッダを新しいFITSファイルに書き出します。**fits.writeto()**を使います。

In [19]:
fits.writeto('bflatn5pa.fits', data, header)   #   p for python 

フラットなんて中間ファイルなので盛りだくさんなヘッダは不要ですよ、という時には、fits.writeto()の引数のheaderを省略すると、最低限必要なヘッダを勝手に作ってくれます。

In [20]:
fits.writeto('bflatn5pa_simple.fits', data)

元のFITSファイルに上書き更新する場合には**fits.update( )**を使います。

In [21]:
data = fits.getdata('bflatn5pa.fits') 
data[:, 2049:] = -999999.  # オーバースキャン部の値を負の大きな値にしておく 
fits.update('bflatn5pa.fits', data, header)

fits.getdata( ) や fits.getheader( )、fits.writeto( )などは簡単で便利 (これらはastropy.io.fitsの中で'convinience functions'と呼ばれている) なのですが、効率の悪いことをしています。その都度にファイルのオープンとクローズをしています。  

その理由から、一般にプログラムコードの中でFITSファイルの読み書きをする場合には、** fits.open() **でファイルを開き、**.headerメソッド**と**.dataメソッド**を使ってヘッダーとデータを読み取ります。ここでは、fits.open()の講習は省略します。興味のある人はadc2017pythonの1回目のほうの資料を参照してください。



### 複数枚のドームフラットからフラットを作成 

IRAFを使わずに、combineします。 numpy.median(), numpy.append()を使います。  

In [22]:
import glob

stack = np.empty((0, 4100, 2080))  #  空の配列を作成

for img in glob.glob('./data1/SUPA003175[0-6]5.fits'):

    imdata = fits.getdata(img)
    omed = np.median(imdata[:, 2049:])  
    imed = np.median(imdata[:, :2048]) 
    imdata = ( imdata - omed ) / (imed - omed)  
            
    stack = np.append(stack, imdata[np.newaxis, :], axis=0)
        
immed = np.median(stack, axis=0)    
immed = immed.astype(np.float32)

fits.writeto('bflatn5p.fits', immed)

#### ターゲットの生データのバイアス引きとフラット割り  
target1の5番フレームの生データを、IRAFを使わずにastropy.io.fitsとnumpyで処理してみます。

In [23]:
imdata = fits.getdata('./data1/SUPA00317705.fits') 
flat = fits.getdata('bflatn5p.fits') 
flat[np.where(flat == 0.0)] = -9999   #   0での割り算を回避
omed = np.median(imdata[:, 2049:])  
imdata = ( imdata - omed ) / flat 
fits.writeto('btarget1n5p.fits', imdata)

### トリミング  

オーバースキャン領域はバイアス値の引き算が終わったあとは不要ですね。  
切り取ってやりましょう。  

In [24]:
imdata = fits.getdata('bflatn5p.fits')
fits.writeto('bflatn5ptrim.fits', imdata[:, :2048])

### 人工的な画像  
ピクセル値が全部ゼロの10 x 10のFITSファイルを作り、いくつかのピクセルにだけ正の値を与えてやります。

In [25]:
imdata = np.zeros((10, 10), dtype='float32')  # 全て0の10x10のndarrayを作成
imdata[0, 1] = 100. #  (x,y)=(1,0)に100を代入。xとyが反転していることに注意
imdata[4, 6] = 50. 

fits.writeto('my10x10.fits', imdata)

出来上がったFITSファイルをDS9で見てみましょう。そして、xとyが反転していることを確かめてください。

### 演習4  

astropy.io.fitsとnumpyを使って以下の処理をしてください。  

(1) './data2/SUPA003175[0-6]2.fits'から2番フレーム用のフラットを作成。  
(2) './data2/SUPA00317882.fits' について、バイアス引き+フラット割りの処理  


### 補足 バイアス値をY方向も考慮に入れる (numpy編)

numpyを使うと、IRAFでやるよりもすっきり書けます。  
まずはフラットを作成します。

In [27]:
imdata = fits.getdata('./data1/SUPA00317505.fits')

bias = np.median(imdata[:, 2049:], axis=1) # X軸に沿ってmedianを計算。biasはサイズ4100の1次元配列 
bias = bias.reshape(4100,1) # ブロードキャスティングできるように整形

imdata = imdata - bias   #  ブロードキャスティングで引き算  

med = np.median(imdata[:, :2045])

imdata = imdata / med 
imdata = imdata.astype(np.float32)

fits.writeto('bflatn5pby.fits', imdata) # ヘッダは生データのものを継承しない

次に、生データもバイアス値のY方向依存を考慮して引きます。

In [31]:
flat = fits.getdata('bflatn5pby.fits')
flat[np.where(flat==0)] = -9999 
tdata = fits.getdata('./data1/SUPA00317705.fits')
bias = np.median(tdata[:, 2049:], axis=1) 
bias = bias.reshape(4100,1) 

tdata = tdata - bias
tdata = tdata / flat 

fits.writeto('btarget1n5pby.fits', tdata) 